In [1]:
import os
print(os.getcwd())
os.chdir("../")
print(os.getcwd())
import tensorflow as tf

e:\Preparation\iNeuron\Full_Stack_Data_Science\Deep_Learning\CNN\Project\CNN_Deep_Classifier\research
e:\Preparation\iNeuron\Full_Stack_Data_Science\Deep_Learning\CNN\Project\CNN_Deep_Classifier


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classess: int

In [3]:
from deepClassifier.constants import *
from deepClassifier.utility import read_yaml_file,create_directories

In [4]:
class ConfigManager:

    def __init__(
        self,
        config_file_path  = CONFIG_FILE_PATH,
        param_file_path  = PARAMS_FILE_PATH
        ):
            self.config = read_yaml_file(config_file_path)
            self.param = read_yaml_file(param_file_path)
            create_directories([self.config.artifact_root])
    
    def get_prepare_basemodel_config(self) -> PrepareBaseModelConfig:
        try:
            config = self.config.prepare_base_model
            param_config = self.param

            create_directories([config.root_dir])

            prepare_Base_Model_Config =   PrepareBaseModelConfig(
                root_dir = Path(config.root_dir),
                base_model_path = Path(config.base_model_path),
                updated_base_model_path = Path(config.updated_base_model_path),
                params_image_size = param_config.IMAGE_SIZE,
                params_learning_rate = param_config.LEARNING_RATE,
                params_include_top = param_config.INCLUDE_TOP,
                params_weights = param_config.WEIGHTS,
                params_classess = param_config.CLASSESS
            )

            return prepare_Base_Model_Config
        except Exception as e:
            raise e


In [26]:
class PrepareBaseModel:

    def __init__(self,config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top= self.config.params_include_top, 
            # Use only CNN layer VGG16 and exclude fully connected layer of VGG16 if its False
        )

        self.save_model(self.config.base_model_path,self.model)

    @staticmethod
    def _prepare_full_model(model,classess,freeze_all,freeze_till,learning_rate_1):
        
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
                for layer in model.layers[:-freeze_till]:
                    model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classess,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(optimizer="SGD" ,
                            loss = "categorical_crossentropy",metrics=["accuracy"])

        full_model.summary()
        return full_model


    def update_base_model(self):
        prepare_full_model = self._prepare_full_model(
                                model = self.model,
                                classess = self.config.params_classess,
                                freeze_all = True,
                                freeze_till = None,
                                learning_rate_1 = self.config.params_learning_rate
                            )

        self.save_model(path=self.config.updated_base_model_path,model=prepare_full_model)

    @staticmethod
    def save_model(path:Path,model):
        model.save(path)

In [27]:
try:
    config = ConfigManager()
    prepareBaseModel = PrepareBaseModel(config.get_prepare_basemodel_config())
    prepareBaseModel.get_base_model()
    prepareBaseModel.update_base_model()
except Exception as e:
    raise e

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   